#**First We get aceess from our google drive**# 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## **Here we import the important libraries for formatting our training data**

In [ ]:
import json
import yaml

In [ ]:
#the yml file gets loaded in the data variable 
with open('gdrive/MyDrive/depression.yml', 'r') as file:
     data = yaml.safe_load(file)

data

In [ ]:
convos = data['conversations']

convos[0]

In [ ]:
output = []

In [ ]:
#Basically here we change the .yml to list of dictionaries
for convo in convos:
  completion = ''
  for i, dialog in enumerate(convo):
    if i == 0:
      prompt = dialog
      prompt = prompt.replace("\xa0", " ")
    else:
      completion += " " + dialog
      completion = completion.replace("\xa0", " ")
  completion = completion.strip()
  line = {'prompt': prompt, 'completion': completion}
  output.append(line)
print(output)

[{'prompt': 'What Is Depression?', 'completion': 'Depression is a common and serious medical illness that negatively affects how you feel, the way you think and how you act. Fortunately,it is also treatable. Depression causes feelings of sadness and/or a loss of interest in activities you once enjoyed. It can lead to a variety of emotional and physical problems and can decrease your ability to function at work and at home.'}, {'prompt': 'I feel i have let my parents down', 'completion': 'No matter what,your parents will always be proud of you and will love you. You will feel much better if you share your feelings with them.'}, {'prompt': 'What are the types of depression?', 'completion': 'There are many different forms of depression but most common types are. Clinical depression,persistent depressive disorder,bipolar disorder,postnatal disorder.'}, {'prompt': 'I am good for nothing.', 'completion': "Don't ever think you cannot achieve what you want. You are capable of reaching your goa

In [ ]:
from google.colab import files

In [ ]:
#Here we convert our output which was a list of dictionaries into a jsonl file
with open('depression-d.jsonl', 'w') as outfile:
        for i in output:
            json.dump(i, outfile)
            outfile.write('\n')


files.download('depression-d.jsonl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install openai

# **Here we prepare our data for fine tuining**

In [ ]:
!openai tools fine_tunes.prepare_data -f depression-d.jsonl



In [ ]:
os.environ['OPENAI_API_KEY'] = '#ENTER YOUR API KEY HERE'

In [ ]:
!set OPENAI_API_KEY=<YOUR-API-KEY>

## **Here we finel tune our model using the text-curie-001 model**

In [ ]:
!openai api fine_tunes.create -t "depression-d_prepared.jsonl" -m text-curie-001

In [ ]:
#Here basically we test our trained model
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

query = input("Give your query:")

response = openai.Completion.create(
                model="text-curie-001",
                prompt="The following is a conversation with a therapist and a user. The therapist is JOY, who uses compassionate listening to have helpful and meaningful conversations with users. JOY is empathic and friendly. JOY's objective is to make the user feel better by feeling heard. With each response, JOY offers follow-up questions to encourage openness and tries to continue the conversation in a natural way. \n\nJOY-> Hello, I am your personal mental health assistant. What's on your mind today?\nUser->"+query+"JOY->",
                temperature=0.89,
                max_tokens=162,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0.6,
                stop=["\n"]
  )
print(response)